# Выбор локации для скважины

Нужно решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Построим модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль.

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самыми высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# 1. Загрузка и подготовка данных

### Описание данных

- *id* — уникальный идентификатор скважины;
- *f0*, *f1*, *f2* — три признака точек;
- *product* — объём запасов в скважине (тыс. баррелей).

In [2]:
import pandas as pd
import numpy as np
from scipy import stats as st
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

geo_data0 = pd.read_csv('/datasets/geo_data_0.csv')
geo_data0

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


In [3]:
geo_data0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [4]:
geo_data1 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data1

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...,...
99995,QywKC,9.535637,-6.878139,1.998296,53.906522
99996,ptvty,-10.160631,-12.558096,5.005581,137.945408
99997,09gWa,-7.378891,-3.084104,4.998651,137.945408
99998,rqwUm,0.665714,-6.152593,1.000146,30.132364


In [5]:
geo_data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [6]:
geo_data2 = pd.read_csv('/datasets/geo_data_2.csv')
geo_data2

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...,...
99995,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,nmxp2,-2.419896,2.417221,-5.548444,51.795253


In [7]:
geo_data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
geo_data2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


В данных нет пропусков. Столбец id можно убрать, он не поможет обучению модели.

In [9]:
data = [geo_data0, geo_data1, geo_data2]
for i in data:
    i = i.drop(['id'], axis=1, inplace=True)

In [10]:
#разбивка на признаки и целевой признак для трех регионов
features0 = geo_data0.drop('product', axis=1)
target0 = geo_data0['product']

In [11]:
target1 = geo_data1['product']
features1 = geo_data1.drop('product', axis=1)

In [12]:
target2 = geo_data2['product']
features2 = geo_data2.drop('product', axis=1)

# 2. Обучение и проверка модели

In [13]:
#Разобъем данные на обучающую и валидационную выборки
features_train0, features_valid0, target_train0, target_valid0 = train_test_split(
    features0, target0, test_size=0.25, random_state=12345)

In [14]:
features_train1, features_valid1, target_train1, target_valid1 = train_test_split(
    features1, target1, test_size=0.25, random_state=12345)

In [15]:
features_train2, features_valid2, target_train2, target_valid2 = train_test_split(
    features2, target2, test_size=0.25, random_state=12345)

In [16]:
# Обучение модели линейной регрессии и подсчет RMSE
model = LinearRegression()
model.fit(features_train0, target_train0)
predictions0 = model.predict(features_valid0)
mse0 = mean_squared_error(target_valid0, predictions0)
rmse0 = mse0 ** 0.5
rmse0

37.5794217150813

In [17]:
print('Средний запас предсказанного сырья:', predictions0.mean())

Средний запас предсказанного сырья: 92.59256778438038


In [18]:
model.fit(features_train1, target_train1)
predictions1 = model.predict(features_valid1)
mse1 = mean_squared_error(target_valid1, predictions1)
rmse1 = mse1 ** 0.5
rmse1

0.893099286775616

In [19]:
print('Средний запас предсказанного сырья:', predictions1.mean())

Средний запас предсказанного сырья: 68.728546895446


In [20]:
model.fit(features_train2, target_train2)
predictions2 = model.predict(features_valid2)
mse2 = mean_squared_error(target_valid2, predictions2)
rmse2 = mse2 ** 0.5
rmse2

40.02970873393434

In [21]:
print('Средний запас предсказанного сырья:', predictions2.mean())

Средний запас предсказанного сырья: 94.96504596800489


### Вывод

Наиболее точный результат (RMSE = 0.89), но с наименьшим средним запасом сырья (68.73 тыс. баррелей) получился для второго региона. Результаты остальных регионов близки друг к другу: RMSE около 40 тыс. баррелей, 93 и 95 тыс. баррелей для 1 и 3 регионов соответственно.

# 3. Подготовка к расчёту прибыли

In [22]:
 # бюджет на разработку скважин в регионе в тыс. рублей
budget = 10000000   
# прибыль с тысячи баррелей сырья в тыс. рублей
profit_by_one_barrel = 450 
# количество скважин, на которых планируется начать разработку
num_wells = 200 

In [23]:
# Минимальный объем сырья для безубыточной разработки новой скважины, в тыс. баррелей
well_volume = budget / num_wells / profit_by_one_barrel
well_volume

111.11111111111111

Наиболее близки средние объемы скважин к теоретическому минимальному объему скважины для безубыточной разработки у первого и третьего регионов.

# 4. Расчёт прибыли и рисков 

In [24]:
#Функция для рассчета прибыли
def revenue(target, preditions):
    probs_sorted = preditions.sort_values(ascending=False)
    selected = target[probs_sorted.index][:num_wells]
    profit = selected.sum() * profit_by_one_barrel - budget
    return profit

In [41]:
state = np.random.RandomState(12345)
num_samples = 1000
n = 500
values0 = []

In [42]:
# Бутстреп для 1 региона
target_valid0 = target_valid0.reset_index(drop=True)
predictions0 = pd.Series(predictions0)
for i in range(num_samples):
    target_subsample = target_valid0.sample(n=n, replace=True, random_state=state)
    pred_subsample = predictions0[target_subsample.index]
    rev = revenue(target_subsample, pred_subsample)
    values0.append(rev)

In [43]:
values0 = pd.Series(values0)
print("Средняя выручка:", values0.mean())

Средняя выручка: 425938.5269105924


In [38]:
confidence_interval = (values0.quantile(0.025), values0.quantile(0.975))
#confidence_interval = st.t.interval(alpha=0.95, df=len(values0)-1, loc=values0.mean(), scale=values0.sem())
print("95%-ый доверительный интервал:", confidence_interval)

95%-ый доверительный интервал: (-102090.09483793723, 947976.3533583689)


In [29]:
print('Риск убытков', (values0<0).mean())

Риск убытков 0.06


In [30]:
# 2 регион
values1 = []
target_valid1 = target_valid1.reset_index(drop=True)
predictions1 = pd.Series(predictions1)
for i in range(num_samples):
    target_subsample = target_valid1.sample(n=n, replace=True, random_state=state)
    pred_subsample = predictions1[target_subsample.index]
    rev = revenue(target_subsample, pred_subsample)
    values1.append(rev)

In [31]:
values1 = pd.Series(values1)
print("Средняя выручка:", values1.mean())

Средняя выручка: 518259.493697325


In [32]:
confidence_interval = (values1.quantile(0.025), values1.quantile(0.975))
#confidence_interval = st.t.interval(alpha=0.95, df=len(values1)-1, loc=values1.mean(), scale=values1.sem())
print("95%-ый доверительный интервал:", confidence_interval)

95%-ый доверительный интервал: (128123.23143308601, 953612.9820669085)


In [33]:
print('Риск убытков', (values1<0).mean())

Риск убытков 0.003


In [34]:
# 3 регион
values2 = []
target_valid2 = target_valid2.reset_index(drop=True)
predictions2 = pd.Series(predictions2)
for i in range(num_samples):
    target_subsample = target_valid2.sample(n=n, replace=True, random_state=state)
    pred_subsample = predictions2[target_subsample.index]
    rev = revenue(target_subsample, pred_subsample)
    values2.append(rev)

In [35]:
values2 = pd.Series(values2)
print("Средняя выручка:", values2.mean())

Средняя выручка: 420194.00534405006


In [39]:
confidence_interval = (values2.quantile(0.025), values2.quantile(0.975))
#confidence_interval = st.t.interval(alpha=0.95, df=len(values2)-1, loc=values2.mean(), scale=values2.sem())
print("95%-ый доверительный интервал:", confidence_interval)

95%-ый доверительный интервал: (-115852.60916001163, 989629.9398445741)


In [37]:
print('Риск убытков', (values2<0).mean())

Риск убытков 0.062


### Вывод

По результатам исследования нам подходит только второй регион, так как только у него риск убытков < 2.5 % и равен 0.3 %. В тоже время во втором регионе наибольшая средняя предполагаемая выручка, равная 518 млн. рублей, которая попадает в 95% доверительный интервал.